In [53]:
import numpy as np
import pandas as pd
import os
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import random
from torch.nn.utils.rnn import pack_padded_sequence
from torch import nn, optim
from torchsummary import summary


In [2]:
def collate_fn(batch):
    padded_batch = []
    max_length = max(sample[0].shape[1] for sample in batch)
#     print('max_length', max_length)
    for sample in batch:
        pad_length = max_length - sample[0].shape[1]
        padded_sample = torch.nn.functional.pad(sample[0], (0, pad_length))
        batch_label = sample[1]
        padded_batch.append((padded_sample,int(batch_label)))
        
#     print('padded_batch',padded_batch)
    return padded_batch
   

In [3]:
class CustomDataset(Dataset):
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.file_list = os.listdir(folder_path)
        self.sample = []
        for i in range(len(self.file_list)):
            file_name = self.file_list[i]
            file_path = os.path.join(self.folder_path, file_name)
            data = torch.load(file_path)  ## nested list of [ file name, tensors ]
            label = self.file_list[i].split('.')[0]
            for tensor in data:
                self.sample.append((tensor[1], label))

    def __len__(self):
        return len(self.sample)   ## dont hard code

    def __getitem__(self, idx):
#         print('self.sample[idx]',self.sample[idx])
        return self.sample[idx]  # returns only samples ???? labels????

In [4]:
train_folder = "mad_trail/train/"
dev_folder = "mad_trail/dev/"
save_path = 'model.pt'
train_dataset = CustomDataset(train_folder)
dev_dataset = CustomDataset(dev_folder)

In [5]:
batch_size =32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [6]:
train_loader = next(iter(train_dataloader))

for i in train_loader:
    break


In [88]:
### Lstm blocks   ******** IMPROVE  *********
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        ### 1D CNN

        self.layer1 = torch.nn.Conv1d(in_channels=input_size, out_channels= hidden_size, kernel_size=5, stride=2)
        self.act1 = torch.nn.ReLU()

        ## BiLSTM
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.lstm_drop = nn.Dropout(0.1)

        # linear 
        self.linear = nn.Linear(hidden_size*2, output_size)  ### BiLSTM
        self.linear_drop = nn.Dropout(0)
        ## softmax layer
        self.activ = nn.Softmax(dim=1)
        
    def forward(self, x):

        # out,_ = self.lstm(x, (h0, c0))

        
        x = self.layer1(x)
        x = self.act1(x)
        
        x = x .transpose(1,2)
        
        out, _ = self.lstm(x)
        out = self.lstm_drop(out)
        out = self.linear(out[:, -1, :])  
        out = self.linear_drop(out)
        out = self.activ(out )
        # out = out.transpose(1, 2).reshape(-1, batch_size).transpose(1, 0)
        return out
    

In [89]:
def train(n_epochs, trainloader, valloader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model.train().to(device)

    val_loss_min = np.inf

    train_losses = []
    val_losses = []

    train_accuracies = []
    val_accuracies = []
    
    for i_epoch in range(n_epochs):
        epoch_train_loss = 0
        epoch_train_acc = 0
        model.train().to(device)
        ## train
        for batch in tqdm(trainloader, desc="Train"):
            opti.zero_grad()
            batch_x = [i[0] for i in batch]
            batch_x = torch.stack(batch_x)
            batch_y = [i[1] for i in batch]
            y_out = model(batch_x)
            y_hat = torch.argmax(y_out, dim=1)

            ##### Doubt?? do i need to send logits(y_out) or y_hat to criterion ??? 
            loss = criterion(y_out, torch.tensor(batch_y))
            loss.backward()
            opti.step()
            acc = torch.sum(y_hat==torch.tensor(batch_y))
            epoch_train_acc+=acc
            epoch_train_loss += loss.item()

    
        epoch_train_loss = epoch_train_loss/len(trainloader)
        epoch_train_acc = epoch_train_acc/len(trainloader)
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        ### val
        model.eval()
        with torch.no_grad():
            epoch_val_loss = 0
            epoch_val_acc = 0
            for batch in tqdm(valloader, desc="Val  "):
                batch_x = [i[0] for i in batch]
                batch_x = torch.stack(batch_x)
                batch_y = [i[1] for i in batch]
                y_out = model(batch_x)
                y_hat = torch.argmax(y_out, dim=1)
                loss = criterion(y_out, torch.tensor(batch_y))
                acc = torch.sum(y_hat==torch.tensor(batch_y))
                epoch_val_acc+=acc
                epoch_val_loss += loss.item()

            epoch_val_loss = epoch_val_loss/len(valloader)
            epoch_val_acc = epoch_val_acc/len(valloader)
            val_losses.append(epoch_val_loss)
            val_accuracies.append(epoch_val_acc)
            
        # to save model
            if epoch_val_loss < val_loss_min:
                torch.save(model.state_dict(), save_path)
                tqdm.write(f'Validation loss reduced from {val_loss_min:.6f} to {epoch_val_loss:.6f}, saving model at {save_path} ...')
                val_loss_min = epoch_val_loss

#         if scheduler:
#             scheduler.step()
#             tqdm.write(f'Updating lr to {scheduler.get_last_lr()}')

        tqdm.write(f'Epoch : {i_epoch+1:02}\nTrain Loss = {epoch_train_loss:.4f}\tTrain Acc = {epoch_train_acc}\n  Val Loss = {epoch_val_loss:.4f}\t  Val Acc = {epoch_val_acc}\n')

    loss_dict = {"train_losses" : train_losses,
                 "val_losses" : val_losses,
                "train_acc" : train_accuracies,
                "val_acc" : val_accuracies}

    return loss_dict



In [91]:
####################################3
it = iter(train_dataloader)
first = next(it)
input_size=first[0][0].shape[0]
# print(f'input_size of {i_epoch}', input_size)
print('input_size', input_size)
hidden_size = 128
num_layers = 4
output_size = 8  # Modify this according to your specific task

n_epochs = 4
criterion = nn.CrossEntropyLoss()

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
print(model)
opti = optim.Adam(model.parameters())
# opti = optim.SGD(model.parameters(),lr=0.01, weight_decay=1e-3, momentum=0.8)

results = train(n_epochs, train_dataloader, dev_dataloader)



input_size 512
LSTMModel(
  (layer1): Conv1d(512, 128, kernel_size=(5,), stride=(2,))
  (act1): ReLU()
  (lstm): LSTM(128, 128, num_layers=4, batch_first=True, bidirectional=True)
  (lstm_drop): Dropout(p=0.1, inplace=False)
  (linear): Linear(in_features=256, out_features=8, bias=True)
  (linear_drop): Dropout(p=0, inplace=False)
  (activ): Softmax(dim=1)
)


Val  : 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.26it/s]


Validation loss reduced from inf to 2.079650, saving model at model.pt ...
Epoch : 01
Train Loss = 2.0804	Train Acc = 3.700000047683716
  Val Loss = 2.0796	  Val Acc = 3.799999952316284



Train:  10%|███████▌                                                                    | 2/20 [00:23<03:31, 11.77s/it]


KeyboardInterrupt: 